<a href="https://colab.research.google.com/github/eecheonwu/ammi_ml-proj/blob/main/AMMI_PROJECT_OPTIMIZATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -U pandas==1.0.3
! pip install -U trains>=0.16.2
! pip install -U optuna==2.0.0

     |████████████████████████████████| 10.0MB 2.8MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.0.3 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement pandas>=1.0.4, but you'll have pandas 1.0.3 which is incompatible.
  Found existing installation: pandas 1.1.2
    Uninstalling pandas-1.1.2:
      Successfully uninstalled pandas-1.1.2


     |████████████████████████████████| 235kB 4.3MB/s 
     |████████████████████████████████| 163kB 6.7MB/s 
     |████████████████████████████████| 81kB 4.9MB/s 
     |████████████████████████████████| 81kB 6.9MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 112kB 12.3MB/s 
     |████████████████████████████████| 143kB 11.9MB/s 
  Created wheel for optuna: filename=optuna-2.0.0-cp36-none-any.whl size=312967 sha256=3cc437639841ffc505b3fcd9e0a84ee2d07ca9204a1bb0c3cc291448d0f7b236
  Stored in directory: /root/.cache/pip/wheels/b5/c9/03/c45484454bf657ffed0ed6af153bd3d213928df115eb2a56eb
  Created wheel for pyperclip: filename=pyperclip-1.8.1-cp36-none-any.whl size=11119 sha256=1952c7c5cdae237cf8fd713067395750326b7009a4c167c8f8e1ffe73ef91a43
  Stored in directory: /root/.cache/pip/wheels/44/10/3a/c830e9bb3db2c93274ea1f213a41fabde0d8cf3794251fad0c
Successfully built optuna pyperclip


In [ ]:
from trains.automation import UniformParameterRange, UniformIntegerParameterRange
from trains.automation import HyperParameterOptimizer
from trains.automation.optuna import OptimizerOptuna

from trains import Task


In [ ]:
task = Task.init(project_name='Hyperparameter Optimization',
                 task_name='Hyperparameter Optimization for AMMI Project',
                 task_type=Task.TaskTypes.optimizer)


TRAINS Task: created new task id=8b0343c7e5f6438fb647362d3e50d02f
2020-10-18 21:49:43,485 - trains.Task - INFO - No repository found, storing script code instead
TRAINS results page: https://demoapp.trains.allegro.ai/projects/e26fd5f55d224d24b4f2f22a9899e262/experiments/8b0343c7e5f6438fb647362d3e50d02f/output/log


In [ ]:
TEMPLATE_TASK_ID = '8b0343c7e5f6438fb647362d3e50d02f'

In [ ]:
optimizer = HyperParameterOptimizer(
    base_task_id=TEMPLATE_TASK_ID,  # This is the experiment we want to optimize
    # here we define the hyper-parameters to optimize
    hyper_parameters=[
        UniformIntegerParameterRange('n_epochs', min_value=5, max_value=30, step_size=1),
        UniformIntegerParameterRange('lr_step', min_value=0, max_value=8, step_size=1),
        UniformParameterRange('n_weight', min_value=0, max_value=0.5, step_size=0.05),
        UniformParameterRange('n_lr', min_value=0.0005, max_value=0.01, step_size=0.0005),
        UniformParameterRange('n_gamma', min_value=0, max_value=0.3, step_size=0.05),
    ],,
           
    ],
    # setting the objective metric we want to maximize/inimize
    objective_metric_title='losses',
    objective_metric_series='loss',
    #objective_metric_sign='min',  # maximize or minimize the objective metric

    # setting optimizer - trains supports GridSearch, RandomSearch, OptimizerBOHB and OptimizerOptuna
    optimizer_class=OptimizerOptuna,
    
    # Configuring optimization parameters
    execution_queue='default',  # queue to schedule the experiments for execution
    max_number_of_concurrent_tasks=3,  # number of concurrent experiments
    optimization_time_limit=30.,  # set the time limit for the optimization process
    compute_time_limit=120,  # set the compute time limit (sum of execution time on all machines)
    total_max_jobs=10,  # set the maximum number of experiments for the optimization. 
                        # Converted to total number of iteration for OptimizerBOHB
    min_iteration_per_job=15000,  # minimum number of iterations per experiment, till early stopping
    max_iteration_per_job=150000,  # maximum number of iterations per experiment
)

2020-10-18 22:12:54,726 - trains.automation.optimization - WARNING - Could not find requested hyper-parameters ['n_epochs', 'n_gamma', 'n_lr', 'n_momentum', 'n_step_size', 'n_weight_decay'] on base task 8b0343c7e5f6438fb647362d3e50d02f
2020-10-18 22:12:55,017 - trains.automation.optimization - WARNING - Could not find requested metric ('', '') report on base task 8b0343c7e5f6438fb647362d3e50d02f


In [ ]:
optimizer.set_report_period(1)  # setting the time gap between two consecutive reports
optimizer.start()  
optimizer.wait()  # wait until process is done
optimizer.stop()  # make sure background optimization stopped

In [ ]:
# optimization is completed, print the top performing experiments id
k = 3
top_exp = optimizer.get_top_experiments(top_k=k)
print('Top {} experiments are:'.format(k))
for n, t in enumerate(top_exp, 1):
    print('Rank {}: task id={} |result={}'
          .format(n, t.id, t.get_last_scalar_metrics()['loss']['loss']['last']))
